<a href="https://colab.research.google.com/github/soumyadip1995/Google-doodle-Sebastian-Bach/blob/master/Google_Doodle_Coconet_Harmonization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction




  ![alt text](https://www.google.com/logos/doodles/2019/celebrating-johann-sebastian-bach-5702425880035328.4-lawcta.gif)

(Img : google)


A few days ago, Google had unveiled for the first time an AI powered doodle. Google had celebrated the 334th birthday of John Sebastian Bach by dedicating the first ever AI powered doodle in his honor.

Made in partnership with the Google Magenta and Google PAIR teams, the Doodle is an interactive experience encouraging players to compose a two measure melody of their choice. With the press of a button, the Doodle then uses machine learning to harmonize the custom melody into Bach’s signature music style(or a Bach 80's rock style hybrid if you happen to find a very special easter egg in the Doodle...:)).


The first step in developing the Doodle is to create a machine learning model to power it. The model used in today's Doodle was developed by Magenta Team AI Resident Anna Huang, who developed **Coconet**: a versatile model that can be used in a wide range of musical tasks—such as harmonizing melodies or composing from scratch .


##Introducing Coconet

It’s hard to write a good melody, let alone counterpoint, where multiple melodic lines need to sound good together. Like every music student, we turned to Bach for help! Using a dataset of 306 chorale harmonizations by Bach, we were able to train machine learning models to generate polyphonic music in the style of Bach.

Coconet is trained to restore Bach’s music from fragments: we take a piece from Bach, randomly erase some notes, and ask the model to guess the missing notes from context. The result is a versatile model of counterpoint that accepts arbitrarily incomplete scores as input and works out complete scores. This setup covers a wide range of musical tasks, such as harmonizing melodies, creating smooth transitions, rewriting and elaborating existing music, and composing from scratch.

Whereas traditional models generate notes in chronological order from beginning to end, Coconet can start anywhere and develop the material in any order. This flexibility makes it attractive as a tool to support the compositional process. One way a musician might incorporate this into their workflow is to repeatedly let Coconet fill out the score, each time keeping the good stuff and erasing the rest. In fact, this is how Coconet works internally: it produces material in a loop, repeatedly rewriting and erasing its own work. It starts with rough ideas, and then goes back and forth to work out details and tweak the material into a coherent whole.




##How does Coconet Work ?





 **COCONET** is a deep convolutional model trained to reconstruct partial scores. Once trained, COCONET provides direct access to all conditionals of the form $p(x_i | x_C)$ where 
 
 C selects a fragment of a musical score x . COCONET addresses the general task of completing partial scores; special cases of this task include ”bridging” two musical fragments, and temporal upsampling and extrapolation.


For our purposes, **“musical scores” ** are three-dimensional objects. The Bach chorales are written for four voices, soprano (S), alto (A), tenor (T) and bass (B). The music for each voice is represented in pianoroll: a two-dimensional array with (discretized) time extending horizontally and pitches laid out vertically. We assumed each voice sings exactly one pitch at any given time. Thus ordinarily for each voice, at each point in time, we have a one-hot pitch vector whose elements are all zero except for a single one indicating the pitch being sung.



![alt text](https://s3.amazonaws.com/halleonard-pagepreviews/HL_DDS_0000000000816380.png)

We treat the stack of pianorolls(input)(as you have noticed in the doodle, img above(Google arts)) as a convolutional featuremap,

with time and pitch forming the two-dimensional convolutional space, and each voice providing a channel. As the scores we will be feeding into the model are incomplete, we supply one additional channel per voice with a mask: binary values indicating at each point in time whether the pitch for that voice is known.


The model is a fairly straightforward convolutional neural network with batch normalization and residual connections. For the doodle, which runs the model in the browser using a Tensorflow.js implementation. (Img: Tensorflow magenta)


![alt text](https://magenta.tensorflow.org/assets/coconet/coconet-figure.png)

What comes out of the model is again a stack of pianorolls, one per voice, but this time containing probability distributions over the pitches of the erased notes. The model uses the notes that were given to try to figure out the notes that were erased, resulting in a categorical distribution over the pitch being sung by each voice at each point in time.


###Putting it into perspective

One way of accounting for these interactions would be to sample one of the pitches, add it to the incomplete score, and pass the result through the model again to recompute the distributions for the remaining pitches. By repeating this process until all pitches have been determined, we complete the score while taking all interactions into account. This sequential sampling procedure expects the model to be able to accurately determine the unknown pitches one by one. This strong assumption often leads these processes off the rails when the model gets increasingly confused by its own creation.

We instead use a more robust procedure: we treat the model’s output as a rough draft, which we gradually refine by repeated rewrites. Specifically, we sample all of the pitches simultaneously, obtaining a complete (but typically nonsensical) score, which is then partially erased and passed into the model again, after which the process repeats. Over time, we erase and rewrite fewer and fewer notes, enabling the process to settle on a coherent result.


##Mathematics and Melody






![alt text](https://static-news.moneycontrol.com/static-mcnews/2019/03/Bach-doodle-653x435.jpg)


To simplify the presentation, we will consider a toy example of modeling a sequence of three variables, $x_1, x_2$  and $x_3$. Concretely, this could be a three-note melody, or a three-tone chord, with each variable taking a pitch as its value. The problem of modeling the sequence of $x_1,x_2,x_3$ is then the problem of representing and learning the joint probability distribution $p(x_1,x_2,x_3)$ of how likely is a given sequence $x_1, x_2, x_3$ to occur in natural data?



This is a hard problem as it is not enough to model the independent (marginal) distributions $p(x_1), p(x_2), p(x_3)$ because the variables interact.

For each possible value of x1, there are conditional distributions$ p(x_2|x_1)$, $p(x_3|x_1)$ on the other variables that depend on the value of $x_1$ (just as the second note in a three-note melody will depend on the first note, and the second tone in a three-tone chord will depend on the first tone). If we have models of both $p(x_1)$ and $p(x_2|x_1)$, we can compose them to obtain a model of$ p(x_1,x_2)=p(x_1)p(x_2|x_1)$. If we also have a model of $p(x_3|x_1,x_2)$, we can compose all three to obtain a model of the desired joint distribution $ p(x_1,x_2,x_3)=p(x_1)p(x_2|x_1)p(x_3|x_1,x_2)$

This is one example of an autoregressive factorization of the joint probability distribution: the hard-to-handle function $p(x_1,x_2,x_3)$ is broken up into autoregressive factors $p(x_1), p(x_2|x_1)$ and $p(x_3|x_1,x_2)$...... eq (1)


###Autoregressive model

In machine learning, an autoregressive model learns from a series of timed steps and takes measurements from previous actions as inputs for a regression model, in order to predict the value of the next time step.




##Introducing NADE



Distribution estimation is one of the most general problems addressed by machine learning.
From a good and flexible distribution estimator, in principle it is possible to solve a variety
of types of inference problem, such as classification, regression, missing value imputation,
and many other predictive tasks.
Currently, one of the most common forms of distribution estimation is based on directed
graphical models. In general these models describe the data generation process as sampling
a latent state h from some prior p(h), followed by sampling the observed data x from some. 


conditional p(x | h). Unfortunately, this approach quickly becomes intractable and requires
approximations when the latent state h increases in complexity. Specifically, computing
the marginal probability of the data, 

$p(x) =\sum_{h}p(x | h) p(h)$,  


is only tractable under fairly
constraining assumptions on p(x | h) and p(h).
 
 A third approach to distribution estimation, based on autoregressive models and feed-forward neural networks is being considered here. This  particular approach is referred to as
**Neural Autoregressive Distribution Estimation (NADE)** 

Its main distinguishing property
is that computing p(x) under a NADE model is tractable and can be computed efficiently,
given an arbitrary ordering of the dimensions of x

## NADE definition


We consider the problem of modeling the distribution p(x) of input vector observations x.
For now, we will assume that the dimensions of x are binary, that is $ x_d ∈ {0, 1} ∀d$. The
model generalizes to other data types.

NADE begins with the observation that any D-dimensional distribution p(x) can be
factored into a product of one-dimensional distributions, in any order o (a permutation of
the integers 1, . . . , D):



$$p(x) = {\prod_{d=1}^{D}}
p(x_{o_{d}})
| x_{o_{<d}} )$$



Here* o<d *contains the first d − 1 dimensions in ordering o and $x_{o_{< d}}$ 

is the corresponding
subvector for these dimensions. Thus, one can define an ‘autoregressive’ generative model of
the data simply by specifying a parameterization of all D conditionals $p(x_{o_{d}})
| x_{o_{<d}} )$


**NADE generally promotes fixed ordering **

##Let us consider a method  of fixed ordering 


The above factorization....eq(1) is the most natural for sequence data, as it follows the order of the sequence. In the context of monophonic music (such as a melody), that means the distribution for each note is informed by the notes leading up to it. This gives us the forward ordering (1,2,3). Another natural factorization is to go backwards (3,2,1): establish the conclusion first, then work away from it. We can represent these graphically as follows(Img: tensorflow Magenta)


![alt text](https://magenta.tensorflow.org/assets/coconet/autoregressive.png)


More generally, there exists an autoregressive factorization for each possible ordering of the variables. In a problem with N variables, there are N! possible orderings. In our three-variable case, we can enumerate all six of them: (Img: tensorflow Magenta)


![alt text](https://magenta.tensorflow.org/assets/coconet/3_factorial_orderings.png)


These all provide viable ways of modeling three-note melodies or three-tone chords. In theory, it makes no difference which ordering you choose. In practice, however, the choice of ordering determines the applicability of the model – you can’t play along in real time with a backwards model – and introduces an inductive bias that has strong  effects on what the model learns. That is, in practice, models trained according to different factorizations end up learning different joint distributions.

Music as we know consists of multiple simultaneous sequences – multiple instruments playing together. In this case there isn’t really a natural ordering of the variables, although there are two obvious ways in which to flatten the multiple sequences. They are shown below, with time running horizontally and instruments laid out vertically:

On the left, we have interleaved the instruments; the ordering is S, A, T, B, S, A, T, B etc. This ordering favors harmony: the model generates the music one chord at a time. On the right, we have concatenated the instrument parts, to give the ordering S, S, S, S, A, A, A, A etc. Now we favor melody, as the model generates one line after another. These two very different perspectives are actually a familiar source of tension in music theory.(Img: tensorflow Magenta)

![alttext](https://magenta.tensorflow.org/assets/coconet/harmony_vs_melody.png)








No type of single ordering works for us – we wish to be able to complete arbitrarily partial scores. A composer may give us only the beginning of a score, or only the end, or both the beginning and end but not the middle. Or one or more instruments may be missing. Or several instruments may be missing material at different measures. We cannot depend on any instrument coming before another, or any measure coming before any other. In fact, we need to be able to support any ordering!
In general, NADE permits any one fixed ordering, and although all orderings are equivalent from a theoretical perspective, they differ in practice due to effects of optimization and approximation.  Hence,
**We need a new technique!!**


##Orderless NADE


 We can train NADE for all orderings o simultaneously using a new technique called the orderless NADE training procedure.

This procedure relies on the observation that, thanks to parameter sharing, computing

$p_θ(x_{o_{d}})
| x_{o_{<d}} )$

will not be computationally expensive for any number of dimensions.
Hence for a given o and d we can simultaneously obtain
partial losses for all orderings that agree with* o* up to *d*:


An orderless NADE model offers **direct access to all distributions** of the form $p_θ(x_i|x_C )$ conditioned on any set
of contextual variables $x_C = x_{o_{< d}}$  that might already be
known. 

This gives us a very flexible generative model;

**For this reason, we can use these conditional distributions to
complete arbitrarily partial musical scores.**

Let's elaborate


##Modeling without any order

As it turns out,  the intuitively motivated training procedure based on inpainting has us covered. When we feed a partially erased score into the model, what comes out can be interpreted as conditionally independent distributions over the erased variables. Let us return to our example of the three-variable sequence x1, x2, x3. Suppose we erase x2 and x3; then the model observes x1 and produces conditional distributions on the values of x2 and x3. (Img: tensorflow Magenta)


![alt text](https://magenta.tensorflow.org/assets/coconet/coconet-conditional.png)


The conditional distributions p(x2|x1) and p(x3|x1) so obtained appear as factors in two of the six orderings of the three variables. In general, depending on which variables we erase, we can compute any conditional factor from any of the orderings. By composing such conditional factors, we can form a model corresponding to any desired ordering. Essentially, the inpainting model provides an ensemble of autoregressive models, one model for each possible ordering!

Moreover, we can train this ensemble much more efficiently than the naive approach of sampling an ordering and evaluating its conditional factors one by one. 






##Training the coconet

To train Coconet, then, we take a training example from our dataset, choose uniformly how many variables to erase, and choose uniformly the particular subset of variables to erase. We feed the partially erased score into the model (along with a mask indicating which variables were erased) and obtain a set of independent distributions over the values of the erased variables. We then compute the log-likelihoods of the true values and average across the erased variables, which corrects for a subtle scaling issue. This gives us the loss, which we minimize using backprop and stochastic gradient descent as usual.


In 2014, Uria et al. proposed Orderless NADE, an orderless version of the Neural Autoregressive Distribution Estimator (NADE) that has exactly the same structure as this model. The difference between Coconet and Orderless NADE is in the process used to sample from the trained model.


##Gibbs Sampling


In statistics, Gibbs sampling or a Gibbs sampler is a Markov chain Monte Carlo (MCMC) algorithm for obtaining a sequence of observations which are approximated from a specified multivariate probability distribution, when direct sampling is difficult. This sequence can be used to approximate the joint distribution (e.g., to generate a histogram of the distribution); to approximate the marginal distribution of one of the variables, or some subset of the variables (for example, the unknown parameters or latent variables); or to compute an integral (such as the expected value of one of the variables). Typically, some of the variables correspond to observations whose values are known, and hence do not need to be sampled. The gibbs sampling algorithm is shown below.



![alt text](https://slideplayer.com/slide/4467056/14/images/6/Gibbs+Sampling+Algorithm.jpg)






##Gibbs Sampling : generating from multiple Orderings




Although Orderless NADE learns an ensemble of orderings, the associated sampling procedure still effectively samples according to a single ordering. Specifically, Uria et al. proposed to uniformly choose an ordering, and then generate the variables one by one according to the chosen ordering. The music is still composed in a single pass, and no iterative improvement takes place.

**Why  it might be hard to write music in a single pass?** 


Say we’re starting from an empty page, and have to write down our first note for a symphony, knowing we can not change it later. This is a tough decision: we have to account for all the possible futures we might be in and this note has to be right. Later on, as more notes are in place, we have more context to inform our decisions. What if we didn’t have to compose from scratch and could always have some context to work with.

**And it turns out we can, by using Gibbs sampling!**

Gibbs sampling is a process that samples from a **joint distribution** by repeatedly resampling individual variables. We use it as a metaphor for repeatedly rewriting parts of a musical score. At every step, we erase parts of the score and let the model rewrite the erased parts. This way, the model always has some context to anchor to. Although the context is itself in flux and likely to be rewritten on later iterations, that’s okay: the model’s current decisions are not set in stone either. Gradually, the score settles into an internally consistent state.

The process just described is more accurately called **blocked Gibbs sampling** because **we resample more than one variable at a time**. 

If you were to visualize the probability distribution as a landscape, you would see peaks at probable configurations, separated by vast valleys of improbable configurations. Mass resampling helps explore the space of possibilities by taking large jumps, whereas resampling one variable at a time tends to settle on a nearby peak.We start by rewriting large portions of the score in order to explore the space, and we gradually rewrite less and less in order to settle on a plausible musical score.




This works as long as we anneal the  size over time, so that eventually we are **sampling one variable at a time conditioned on all the other variables**, as in the basic Gibbs sampling algorithm. While *Yao et al.* thought of this as a faster approximation to the original Orderless NADE sampling procedure, in the original paper they have  shown empirically that for  use case it actually improves the quality of the samples.

##The result

![alt text](https://e3.365dm.com/19/03/768x432/Ut_HKthATH4eww8X4xMDoxOjA4MTsiGN_4615184.jpg)

Once the model is trained, we have several ways of extracting music from the probability distributions produced by the model. We can sample each of the pitches simultaneously according to its distribution.

An example of Coconet harmonizing melodies from Bach chorales that were not included in the training set is shown below.. The first is the original chorale melodies and  the second is the Coconet’s harmonization. In the original Doodle we can shift chord and harmonize as we see fit. Check out the example below.



In [0]:
Original Chorale



In [0]:
Coconet Harmonization

In [0]:
Coconet compostion from scratch

Google has also built an expanded interface, called [Cuocou](https://coconet.glitch.me/#67:3:0,67:4:0,67:19:0,67:20:0,65:2:0,65:5:0,65:18:0,65:21:0,64:0:0,64:1:0,64:6:0,64:11:0,64:12:0,64:16:0,64:17:0,64:22:0,64:27:0,62:7:0,62:10:0,62:14:0,62:23:0,62:26:0,62:28:0,61:30:0,60:8:0,60:9:0,60:24:0,60:25:0)  that allows more general interaction with Coconet. It enables a new type of collaborative composition with AI, where you can iteratively improve a musical piece by erasing parts that are unsatisfying and asking the model to infill again. You can click infill repeatedly to get different variations. Try out the examples below or start by splashing random notes on a page.


